# Tutorial 5: Monte Carlo simulation of the 2D Ising model

## Physics 7810, Spring 2020

## T5.1 - Monte Carlo simulation of the 2D Ising model

The simplest and most extensively studied lattice model in statistical physics is the *Ising model*, which provides a minimal theoretical description of ferromagnetic and antiferromagnetic phases, binary alloys, and lattice gas models of liquid-vapor coexistence.

The Ising model consists of a lattice of $N$ sites $i$ with a single, two-state degree of freedom $s_i$ on each site that can take values $\pm 1$

The Hamiltonian (energy) of the Ising model is

$$
H = - J \sum_{\langle i j \rangle} s_i s_j - B \sum_i s_i,
$$

where the first sum is over all pairs $\langle ij \rangle$ of nearest-neighbor sites, $J$ is the *coupling* between these sites, and $B$ is the *applied field*.

The properties of the Ising model depend critically on the *spatial dimensionality* $d$, and less critically on the *lattice symmetry* (e.g., square vs. triangular), which determines the number of nearest neighbors (coordination number) $z$. There are exact solutions for the 1D and 2D Ising model, but only numerical solutions for the 3D Ising model. Positive coupling $J$ leads to *ferromagnetic* ordering at low temperatures, while negative $J$ favors an *antiferromagnetic* ground state.

Write a Monte Carlo program to simulate the 2D Ising model on a square lattice with periodic boundary conditions. Implement the single spin flip Metropolis algorithm for sampling, and assume that the external field $B$ is zero for now.

Measure and plot the average energy $\langle E \rangle$ and magnetization per spin $\langle M / N \rangle$, at several temperatures $T$. The total Ising model magnetization is $M = \sum_i s_i$.

Determine the critical temperature $T_c$ (the temperature where the magnetization goes to zero with increasing $T$. Hint: you should obtain $T_c \approx 2.27$

Pro tips:
* Save computation time by avoiding unnecessary reevaluation of the exponential function. To achieve this, use an array to store the possible probabilities for spin-flip acceptance.
* Organize your simulation in terms of Monte Carlo *sweeps*, where a sweep is defined as one attempted spin flip per site. You should pick spins for attempted flips at random, to ensure detailed balance.